# La organización Kiva de préstamos entre particulares

### Disponible en Kaggle en:
https://www.kaggle.com/kiva/data-science-for-good-kiva-crowdfunding/version/5

El conjunto de datos elegido recoge estadísticas de Kiva entre 2014 y 2017. Kiva abre un nuevo mundo de oportunidades para los menos favorecidos y nos permite a cualquiera de nosotros convertirnos en superhéroes. Se trata de una organización sin ánimo de lucro que ofrece pequeños préstamos para ayudar a las comunidades desatendidas que no tienen acceso a los servicios bancarios normales. Proporciona una plataforma y une a personas que estén dispuestas a prestar, un mínimo de 25 dólares, y prestatarios que expongan sus necesidades, la finalidad del dinero y las condiciones de devolución,  porque no es una donación, es un préstamo.

Al crear este servicio, Kiva habilita una solución que desbloquea capital para todos y mantiene un interés financiero muy bajo para los prestatarios. Por otro lado, permite que cualquiera sea parte de la solución y brinda a las personas un amplio abanico de opciones para elegir quién, dónde, cuánto y para qué sector desea ayudar.

### Variables y significado

Las variables utilizadas para describir cada préstamo son:


* id: ID único para préstamo
* funding_amount: la cantidad desembolsada por Kiva al agente de campo (USD)
* loan_amount: la cantidad desembolsada por el agente de campo al prestatario (USD)
* country_code: código ISO del país en el que se desembolsó el préstamo
* activity: Categoría más granular
* sector: Categoría de alto nivel
* use: Uso exacto del monto del préstamo
* country: Nombre completo del país en el que se desembolsó el préstamo
* region: Nombre completo de la región dentro del país
* currency: La moneda en que se desembolsó el préstamo
* partner_id: ID de la organización asociada
* posted_time: Hora a la que el agente de campo (intermediario) publica el préstamo en Kiva
* disbursed_time: Hora en que el agente de campo (intermediario) entrega el préstamo al beneficiario
* funded_time: El momento en que el préstamo publicado en Kiva es financiado por los prestamistas por completo
* term_in_months: La duración por la cual el préstamo se desembolsó en meses
* lender_count: El número total de prestamistas que contribuyeron a este préstamo.
* tags: Etiquetas para describir el caso específico
* borrower_genders: letras M, F separadas por comas, donde cada instancia representa un solo hombre / mujer en el grupo
* repayment_interval: Estado de pago
* date: fecha en la base de datos de esta operación.

**Nombre completo del alumno:**  Sara Arribas Martín

**INSTRUCCIONES**: en cada celda debes responder a la pregunta formulada, asegurándote de que el resultado queda guardado en la(s) variable(s) que por defecto vienen inicializadas a `None`. No se necesita usar variables intermedias, pero puedes hacerlo siempre que el resultado final del cálculo quede guardado exactamente en la variable que venía inicializada a None (debes reemplazar None por la secuencia de transformaciones necesarias, pero nunca cambiar el nombre de esa variable). **No olvides borrar la línea *raise NotImplementedError()* de cada celda cuando hayas completado la solución de esa celda y quieras probarla**.

Después de cada celda evaluable verás una celda con código. Ejecútala (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no. Asegúrate de que, al menos, todas las celdas indican que el código es correcto antes de enviar el notebook terminado.

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
# Create a DataFrame using SparkSession
spark = (SparkSession
 .builder
 .appName("ModeloD")
 .getOrCreate())

### Sobre el dataset kiva_loans.csv se pide:

<div class="alert alert-block alert-success">
<b>PREGUNTA:</b>
</div>

**(1 punto)** Ejercicio 1

* Leerlo **sin intentar** que Spark infiera el tipo de dato de cada columna
* Puesto que existen columnas que contienen una coma enmedio del valor, en esos casos los valores vienen entre comillas dobles. Spark ya contempla esta posibilidad y puede leerlas adecuadamente **si al leer le indicamos las siguientes opciones adicionales** además de las que ya sueles usar: `.option("quote", "\"").option("escape", "\"")`.
* Asegúrate de que las **filas que no tienen el formato correcto sean descartadas**, indicando también la opción `mode` con el valor `DROPMALFORMED` como vimos en clase.
* Crear un nuevo DF `kivaRawNoNullDF` en el que se hayan eliminado todas las filas que tengan algún valor nulo en cualquier columna **excepto en la columna tags**, que no será relevante para el análisis y por tanto podemos ignorar sus valores nulos y mantener dichas filas.

In [7]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
kivaRawDF = spark.read.format("csv")\
                 .option("mode", "DROPMALFORMED")\
                 .option("header", "true")\
                 .option("quote", "\"").option("escape", "\"")\
                 .csv("C:/Users/sara.arribas/Documents/Big Data/EjerciciosPostSpark/Teoria y ejercicios/Ejercicios/ModeloD kiva_loans.csv")

# Descomentar estas líneas para calcular la lista de columnas que sí deben tenerse en cuenta para quitar nulos. Después
# tendrás que utilizar dicha lista en la operación que elimina los nulos

columnasExceptoTags = kivaRawDF.columns
columnasExceptoTags.remove("tags")

kivaRawNoNullDF = kivaRawDF.na.drop(subset = columnasExceptoTags)


In [8]:
from pyspark.sql.types import DoubleType
assert(kivaRawNoNullDF.count() == 574115)

<div class="alert alert-block alert-success">
<b>PREGUNTA:</b>
</div>

**(1 punto)** Ejercicio 2

* Las columnas `posted_time` y `disbursed_time` son en realidad instantes de tiempo que Spark debería procesar como timestamp. Partiendo de `kivaRawNoNullDF`, reemplaza **ambas columnas** por su versión convertida a timestamp, utilizando `withColumn` con el mismo nombre de cada columna, y donde el nuevo valor de la columna viene dado por el siguiente código:

        F.from_unixtime(F.unix_timestamp('nombreColumna', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))

* Además, convierte a `DoubleType` la columna `loan_amount` y a `IntegerType` la columna `term_in_months`.

* El DF resultante de todas estas operaciones debe quedar almacenado en la variable `kivaDF`, **cacheado**.


In [13]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

kivaDF = kivaRawNoNullDF.withColumn("posted_time", F.from_unixtime(F.unix_timestamp('posted_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                        .withColumn("disbursed_time", F.from_unixtime(F.unix_timestamp('disbursed_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                        .withColumn("loan_amount", F.col("loan_amount").cast(DoubleType()))\
                        .withColumn("term_in_months", F.col("term_in_months").cast(IntegerType()))\
                        .cache()         

In [14]:
typesDict = dict(kivaDF.dtypes)
assert(typesDict["posted_time"] == "timestamp") 
assert(typesDict["disbursed_time"] == "timestamp") 
assert(typesDict["loan_amount"] == "double") 
assert(typesDict["term_in_months"] == "int")
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
nullsRow = kivaDF.select(cnt_cond(F.col("posted_time").isNull()).alias("posted_nulls"),
              cnt_cond(F.col("disbursed_time").isNull()).alias("disbursed_nulls")).head()
assert(nullsRow.posted_nulls == 0)
assert(nullsRow.disbursed_nulls == 0)

<div class="alert alert-block alert-success">
<b>PREGUNTA:</b>
</div>

**(2 puntos)** Ejercicio 3

Partiendo de `kivaDF`:

* Primero, añade una nueva columna `dias_desembolso` que contenga el número de días que han pasado entre la fecha en que los prestamistas aceptaron financiar un proyecto, y la fecha en que el agente de campo entregó los fondos al beneficiario. Para ello, utiliza `withColumn` en combinación con la función `F.datediff("colFuturo", "colPasado")`
* De manera análoga, añade otra nueva columna `dias_aceptacion` que contenga el número de días entre el anuncio de la necesidad de préstamo y la aceptación de financiarlo por parte de algún prestamista.
* Reemplazar la columna `sector` por otra en la que se hayan traducido las categorías "Education" por "Educacion" (sin tilde para evitar posibles problemas) y "Agriculture" por "Agricultura", dejando como están el resto de categorías. **La sustitución no debe tener más que tres casos**: uno para cada categoría que vamos a reemplazar, y un tercero para el resto de categorías, que deben quedarse como estaban.
* El resultado debe quedar guardado en la variable `kivaTiemposDF`.

In [57]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
# imports......
kivaTiemposDF = kivaDF.withColumn("dias_desembolso", F.datediff("posted_time", "disbursed_time"))\
                      .withColumn("dias_aceptacion", F.datediff("funded_time", "posted_time"))\
                      .withColumn("sector", F.when(F.col("sector") == "Education","Educacion")\
                                            .when(F.col("sector") == "Agriculture","Agricultura")\
                                            .otherwise(F.col("sector")))

tiempos = kivaTiemposDF.toPandas()
tiempos

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,...,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date,dias_desembolso,dias_aceptacion
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,...,2013-12-17 09:00:00,2014-01-02 10:06:32+00:00,12,12,None,female,irregular,2014-01-01,15,1
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,...,2013-12-17 09:00:00,2014-01-02 09:17:23+00:00,11,14,None,"female, female",irregular,2014-01-01,15,1
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,...,2013-12-17 09:00:00,2014-01-01 16:01:36+00:00,43,6,"user_favorite, user_favorite",female,bullet,2014-01-01,15,0
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,...,2013-12-24 09:00:00,2014-01-01 13:00:00+00:00,11,8,None,female,irregular,2014-01-01,8,0
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,...,2013-12-17 09:00:00,2014-01-01 19:18:51+00:00,14,16,None,female,monthly,2014-01-01,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574110,1340272,75.0,75.0,Livestock,Agricultura,Pretend the issue with spanish loan was addres...,MX,Mexico,Iztacalco,MXN,...,2017-07-21 09:00:00,2017-07-22 12:50:43+00:00,13,3,None,female,monthly,2017-07-21,1,0
574111,1340253,25.0,25.0,Livestock,Agricultura,"[True, u'para compara: cemento, arenya y ladri...",PY,Paraguay,Concepción,USD,...,2017-07-21 09:00:00,2017-07-25 00:48:32+00:00,13,1,None,female,monthly,2017-07-21,0,4
574112,1340256,25.0,25.0,Livestock,Agricultura,Reviewed loan use in english.,PK,Pakistan,Attock,PKR,...,2017-07-21 09:00:00,2017-07-22 01:23:18+00:00,13,1,None,female,monthly,2017-07-21,0,1
574113,1340267,25.0,25.0,Livestock,Agricultura,Reviewed loan use in english.,PK,Pakistan,Attock,PKR,...,2017-07-21 09:00:00,2017-07-22 13:07:29+00:00,13,1,None,female,monthly,2017-07-21,0,1


In [30]:
assert(kivaTiemposDF.where("sector == 'Agricultura'").count() == 157003)
assert(kivaTiemposDF.where("sector == 'Educacion'").count() == 28417)
# Comprobamos que las 13 restantes se mantienen sin cambios
assert(kivaTiemposDF.groupBy("sector").count().join(kivaDF.groupBy("sector").count(), ["sector", "count"]).count() == 13)

<div class="alert alert-block alert-success">
<b>PREGUNTA:</b>
</div>

**(3 puntos)** Ejercicio 4

Partiendo de `kivaTiemposDF`, crear un nuevo DataFrame llamado `kivaAgrupadoDF` que tenga:

* Tantas filas como **países (`country`; no usar el código de país)**, y tantas columnas como **sectores** (cada una llamada como el sector) más una (la columna del país, que debe aparecer en primer lugar). En cada celda deberá ir el número **medio (redondeado a 2 cifras decimales)** de días transcurridos en ese país y sector *entre la fecha en que se anuncia la necesidad de préstamo y la fecha en la que un prestamista acepta financiarlo*. Esta columna ha sido calculada en la celda precedente.
* Después de esto, añadir una columna adicional `transcurrido_global` con el número **medio (redondeado a 2 cifras decimales) de días transcurridos en cada país** entre ambas fechas *sin tener en cuenta el sector*. Cada fila tendrá la media de las 15 columnas del apartado anterior.
* Por último, ordenar el DF resultante **descendentemente** en base al tiempo medio global, `transcurrido_global`. El DF resultado de la ordenación debe ser almacenado en la variable `kivaAgrupadoDF`. 

PISTA: utiliza el método `pivot` con el sector para el primer apartado, envolviendo a la función de agregación con la función `F.round`, es decir, `F.round(F.funcionAgregacion(....), 2)`, y `withColumn` con una operación aritmética entre columnas en el segundo. **No debe utilizarse la función `when`** ya que Spark es capaz de hacer directamente aritmética entre objetos columna. La operación aritmética también debe estar envuelta por round: `F.round(op. aritmética entre objetos columna, 2)`.

In [55]:
sectores = ["Services", "Wholesale","Food", "Entertainment", "Housing", "Health", "Clothing", "Personal Use",\
            "Construction", "Arts", "Agricultura", "Educacion", "Retail", "Manufacturing", "Transportation"]

def sumando(lista): 
    suma = 0
    for colName in lista: 
        suma = suma + F.col(colName)
    return suma

kivaAgrupadoDF = kivaTiemposDF.groupBy("country").pivot("sector").agg(F.round(F.mean(F.col("dias_aceptacion")),2))\
                              .withColumn("transcurrido_global", F.round(sumando(sectores) / 15, 2))\
                              .orderBy(F.col("transcurrido_global").desc())

    

panda2 = kivaAgrupadoDF.toPandas()
panda2

,country,Agricultura,Arts,Clothing,Construction,Educacion,Entertainment,Food,Health,Housing,Manufacturing,Personal Use,Retail,Services,Transportation,Wholesale,transcurrido_global
0,United States,12.00,16.76,19.17,14.77,15.21,23.88,17.71,18.26,27.55,20.50,36.50,23.09,21.60,19.60,27.50,20.94
1,Bolivia,20.33,11.40,24.57,22.26,13.60,13.70,23.49,15.94,26.45,12.69,25.25,24.23,23.13,23.51,19.33,19.99
2,Armenia,23.43,10.84,27.76,18.35,10.75,14.38,22.57,18.63,26.96,11.51,21.11,25.98,20.94,25.33,20.14,19.91
3,Tajikistan,20.87,6.69,24.12,19.01,11.79,21.22,21.35,10.06,24.21,8.80,23.69,25.08,14.87,26.70,10.50,17.93
4,Colombia,20.37,8.33,23.69,19.46,6.17,12.93,21.44,10.92,24.34,9.94,25.20,24.63,19.53,25.99,14.83,17.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Burkina Faso,16.11,8.75,11.00,13.10,NaN,NaN,10.19,7.25,NaN,7.52,12.50,10.57,12.53,5.00,NaN,NaN
77,South Africa,6.86,6.00,7.50,6.43,9.90,NaN,8.36,3.50,7.43,7.17,NaN,8.73,5.41,11.13,20.00,NaN
78,Vanuatu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.50,NaN,NaN,NaN,NaN,NaN
79,Moldova,9.87,8.00,12.50,15.33,3.00,12.67,12.09,4.00,22.35,4.50,16.00,15.50,14.06,19.00,NaN,NaN


In [54]:
r1 = kivaAgrupadoDF.head()
assert(r1.country == "United States")
assert((r1.Agricultura - 12.0 < 0.01) | (r1.Agricultura - 12.17 < 0.01))
assert((r1.Educacion - 15.21 < 0.01) | (r1.Educacion - 15.33 < 0.01))
assert(r1.Wholesale - 27.5 < 0.01)
assert((r1.transcurrido_global - 20.94 < 0.01) | (r1.transcurrido_global - 21.04 < 0.01))

<div class="alert alert-block alert-success">
<b>PREGUNTA:</b>
</div>

**(3 puntos)** Ejercicio 5

Partiendo de nuevo de `kivaTiemposDF`, añadir las siguientes columnas:

* Primero, tres columnas adicionales llamadas `transc_medio`, `transc_min`, `transc_max` que contengan, respectivamente, **el número de días medio, mínimo y máximo transcurrido para proyectos de ese mismo país y ese mismo sector** entre la fecha en que se postea la necesidad de préstamo y la fecha en la que alguien acepta financiarlo (es decir, la columna `dias_aceptacion` calculada antes y utilizada también en la celda anterior). Es decir, queremos una columna extra para que podamos tener, junto a cada préstamo, información agregada de los préstamos similares, entendidos como aquellos del mismo país y del mismo sector. **No se debe utilizar JOIN sino solo funciones de ventana**.
* Finalmente, crear otra columna adicional `diff_dias` que contenga la **diferencia en días entre los días que transcurrieron en este proyecto y la media de días de los proyectos similares** (calculada en el apartado anterior). Debería ser lo primero menos lo segundo, de forma que un número positivo indica que este préstamo tardó más días en ser aceptado que la media de días de este país y sector, y un número negativo indica lo contrario. El resultado debe obtenerse aplicando operaciones aritméticas con columnas existentes, **sin utilizar `when`**.
* El DF resultante con las 4 columnas nuevas que hemos añadido debe quedar almacenado en la variable `kivaExtraInfoDF`.

In [67]:
from pyspark.sql import Window

windowPaisSector = Window().partitionBy("country","sector")

kivaExtraInfoDF = kivaTiemposDF.withColumn("transc_medio",F.mean(F.col("dias_aceptacion")).over(windowPaisSector))\
                               .withColumn("transc_min",F.min(F.col("dias_aceptacion")).over(windowPaisSector))\
                               .withColumn("transc_max",F.max(F.col("dias_aceptacion")).over(windowPaisSector))\
                               .withColumn("diff_dias", F.col("dias_aceptacion")-F.col("transc_medio"))

extraInfo = kivaExtraInfoDF.select("transc_medio", "transc_min", "transc_max","diff_dias", "country", "sector").toPandas()
extraInfo

,transc_medio,transc_min,transc_max,diff_dias,country,sector
0,11.000000,0,35,24.000000,Burkina Faso,Clothing
1,11.000000,0,35,-6.000000,Burkina Faso,Clothing
2,11.000000,0,35,20.000000,Burkina Faso,Clothing
3,11.000000,0,35,-6.000000,Burkina Faso,Clothing
4,11.000000,0,35,-8.000000,Burkina Faso,Clothing
...,...,...,...,...,...,...
574110,9.985915,0,36,-1.985915,Timor-Leste,Food
574111,9.985915,0,36,-1.985915,Timor-Leste,Food
574112,9.985915,0,36,-1.985915,Timor-Leste,Food
574113,9.985915,0,36,1.014085,Timor-Leste,Food


In [64]:
r = kivaExtraInfoDF.where("id = '658540'").head()
assert(r.country == 'Burkina Faso')
assert(r.transc_medio - 11.02 < 0.05)
assert(r.dias_aceptacion == 35)
assert(r.diff_dias - 24.0 < 0.001)